In [1]:
!pip install ta
!pip install python-binance

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29106 sha256=17256e746a874f8235a80cc5b2d48bfb47df4f7fc6858acdc20d55b085d1abc8
  Stored in directory: c:\users\lysyi\appdata\local\pip\cache\wheels\45\06\ca\92c1cdaf57f0c88b2832cc67cea98beeea9fc5d2e5310dfcc2
Successfully built ta



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 67.6/67.6 kB 406.9 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 3.2 MB/s eta 0:00:00
     -------------------------------------- 324.5/324.5 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 41.4/41.4 kB 662.3 kB/s eta 0:00:00
     ---------------------------------------- 96.4/96.4 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 61.8/61.8 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 274.4/274.4 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ray 1.4.0 requires aiohttp-cors, which is not installed.
ray 1.4.0 requires aioredis, which is not installed.
ray 1.4.0 requires msgpack<2.0.0,>=1.0.0, which is not installed.
ray 1.4.0 requires py-spy>=0.2.0, which is not installed.
ray 1.4.0 requires redis>=3.5.0, which is not installed.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from binance.client import Client
import pandas as pd
import ta
from binance.exceptions import BinanceAPIException
from time import sleep

In [4]:
client = Client(api_key=api_key, api_secret=secret)

In [220]:
def klines_1m_240m(symbol):
    try:
        df = pd.DataFrame(client.get_historical_klines(symbol, '1m', '240m UTC'))
        df = df.iloc[:,:6]
        df.columns =['Time','Open','High','Low','Close','Volume']
        df = df.set_index('Time')
        df.index = pd.to_datetime(df.index, unit='ms')
        df = df.astype(float)
        return df
    except BinanceAPIException as e:
        print(e)

def klines_1s_60s(symbol):
    try:
        df = pd.DataFrame(client.get_historical_klines(symbol, '1s', '60s UTC'))
        df = df.iloc[:,:6]
        df.columns =['Time','Open','High','Low','Close','Volume']
        df = df.set_index('Time')
        df.index = pd.to_datetime(df.index, unit='ms')
        df = df.astype(float)
        return df
    except BinanceAPIException as e:
        print(e)
        
def strategy(symbol, qty, open_position=False):
    
    while True:
        df = klines_1m_240m(symbol)
        if not open_position:
            if df.iloc[-1].Close <= df.Close.mean()*0.998:
                order = client.create_order(symbol=symbol, side = 'BUY', type ='MARKET', quantity = qty)
                print(order)
                open_position = True
                buyprice = float(order['fills'][0]['price'])
                break
                
    if open_position:
        while True:
            df = klines_1s_60s(symbol)
            if df.iloc[-1].Close >= buyprice*1.003:
                order = client.create_order(symbol=symbol, side = 'SELL', type ='MARKET', quantity = qty)
                print(order)
                sellprice = float(order['fills'][0]['price'])
                break
                
    print([buyprice, sellprice, sellprice-buyprice])
    print([k*qty for k in [buyprice, sellprice, sellprice-buyprice]])
    print('commission ' + str(sellprice*qty*0.001))
    print('profit ' + str(sellprice*qty*0.999-buyprice*qty))
    return sellprice*qty*0.999-buyprice*qty

In [ ]:
amount = 0.47468361200002107+0,36

for i in range(50000):
    try:
        amount += strategy(symbol='BTCUSDT', qty=0.0038)
    except:
        amount += strategy(symbol='BTCUSDT', qty=0.0037)
        
    print('AMOUNT ' + str(amount))

{'symbol': 'BTCUSDT', 'orderId': 21058430182, 'orderListId': -1, 'clientOrderId': 'FCnU1ip665fZpjBDVHSNsO', 'transactTime': 1683473320479, 'price': '0.00000000', 'origQty': '0.00380000', 'executedQty': '0.00380000', 'cummulativeQuoteQty': '110.00532600', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'workingTime': 1683473320479, 'fills': [{'price': '28948.77000000', 'qty': '0.00380000', 'commission': '0.00000380', 'commissionAsset': 'BTC', 'tradeId': 3109276668}], 'selfTradePreventionMode': 'NONE'}
